# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig, Model
import os
import joblib
import logging
import json
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [22]:
ws = Workspace.from_config()
print('Workspace Name: ' + ws.name,
      'Resource group: ' + ws.resource_group,
      'Azure Region: ' + ws.location,
      'Subscription Id: ' + ws.subscription_id, sep = '\n')

Workspace Name: quick-starts-ws-141698
Resource group: aml-quickstarts-141698
Azure Region: southcentralus
Subscription Id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f


In [23]:
# choose a name for experiment
experiment_name = 'Framingham-Experiment'

experiment=Experiment(ws, experiment_name)
run = experiment.start_logging()

In [24]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "Compute-Cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded.........................................................................................

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Framingham"
description_text = "Frahmingham Heart Disease Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Haris-09/Coronary-Heart-Disease-Prediction-Capstone-Project/master/starter_file/framingham.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

In [ ]:
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
                   "max_concurrent_iterations": 4,
                   "primary_metric" : 'accuracy',
                   "verbosity": logging.INFO}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="TenYearCHD",   
                             path = './',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------

{'runId': 'AutoML_6ebf3996-cf8f-42a4-9bd0-9732d46ccf65',
 'target': 'Compute-Cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-31T16:12:03.612896Z',
 'endTimeUtc': '2021-03-31T16:35:01.325079Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'Compute-Cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Framingham-Experiment","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-141698","workspace_name":"quick-starts-ws-141698","region":"southcentralus","compute_target":"Compute-Cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":nul

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_automl_run, best_automl_model = remote_run.get_output()

print('Best Run Id: ' + best_automl_run.id,
     'Best Model Name: ' + best_automl_run.properties['model_name'],
     )

Best Run Id: AutoML_6ebf3996-cf8f-42a4-9bd0-9732d46ccf65_3 Best Model Name: AutoML6ebf3996c3


In [12]:
best_run_metrics = best_automl_run.get_metrics()

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name,":" , metric)

average_precision_score_weighted : 0.7768787320960463
average_precision_score_macro : 0.5459431627238437
log_loss : 0.42257764232117595
f1_score_macro : 0.4588892190256147
precision_score_weighted : 0.719420921315527
AUC_micro : 0.8737676584972626
average_precision_score_micro : 0.8387114367332881
matthews_correlation : 0.0
f1_score_micro : 0.8481155819555264
f1_score_weighted : 0.7784527258598235
norm_macro_recall : 0.0
balanced_accuracy : 0.5
accuracy : 0.8481155819555264
recall_score_macro : 0.5
precision_score_micro : 0.8481155819555264
precision_score_macro : 0.4240577909777632
AUC_macro : 0.5957664141241882
weighted_accuracy : 0.9686688438245291
recall_score_micro : 0.8481155819555264
recall_score_weighted : 0.8481155819555264
AUC_weighted : 0.5957664141241882
accuracy_table : aml://artifactId/ExperimentRun/dcid.AutoML_6ebf3996-cf8f-42a4-9bd0-9732d46ccf65_3/accuracy_table
confusion_matrix : aml://artifactId/ExperimentRun/dcid.AutoML_6ebf3996-cf8f-42a4-9bd0-9732d46ccf65_3/confusio

In [13]:
#TODO: Save the best model
automodel = best_automl_run.register_model(model_name='best_automl_model', 
                                    model_path='outputs/model.pkl',
                                    tags={'Method':'AutoML'},
                                    properties={'Accuracy': best_run_metrics['accuracy']})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
# Download scoring file 
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

# Download environment file
best_automl_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [15]:
from azureml.core.webservice import AciWebservice, Webservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               description='Framingham Heart Disease Prediction with AutoML')

In [16]:
from azureml.core.model import InferenceConfig, Model
inference_config = InferenceConfig(entry_script="score.py", environment=best_automl_run.get_environment())

service = Model.deploy(workspace=ws, 
                       name='automl-webservice', 
                       models=[automodel], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

In [17]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-31 16:38:55+00:00 Creating Container Registry if not exists.
2021-03-31 16:38:55+00:00 Registering the environment.
2021-03-31 16:38:56+00:00 Use the existing image.
2021-03-31 16:38:57+00:00 Submitting deployment to compute..
2021-03-31 16:39:16+00:00 Checking the status of deployment automl-webservice..
2021-03-31 16:41:48+00:00 Checking the status of inference endpoint automl-webservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [18]:
print("Service State: ",service.state)
print("Scoring URI: ",service.scoring_uri)
print("Swagger URI: ",service.swagger_uri)

Service State:  Healthy
Scoring URI:  http://4344bb92-6df2-465b-b3cb-b5f01b4707c4.southcentralus.azurecontainer.io/score
Swagger URI:  http://4344bb92-6df2-465b-b3cb-b5f01b4707c4.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [19]:

data = {
    "data": [
        {
            'male': 0,
            'age': 55,
            'education': 4 ,
            'currentSmoker': 1,
            'cigsPerDay': 20,
            'BPMeds': 0,
            'prevalentStroke': 0,
            'prevalentHyp': 0,
            'diabetes': 0,
            'totChol': 285,
            'sysBP': 175,
            'diaBP': 84,
            'BMI': 25.50,
            'heartRate': 93,
            'glucose': 87
        },
        {
            'male': 1,
            'age': 40,
            'education': 4 ,
            'currentSmoker': 0,
            'cigsPerDay': 30,
            'BPMeds': 0,
            'prevalentStroke': 0,
            'prevalentHyp': 0,
            'diabetes': 0,
            'totChol': 228,
            'sysBP': 121,
            'diaBP': 110,
            'BMI': 30.30,
            'heartRate': 80,
            'glucose': 80
        }
    ]
}

data_sample = json.dumps(data)

In [20]:
scoring_uri = service.scoring_uri
input_data = data_sample

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
response = requests.post(scoring_uri, input_data, headers=headers)
response.json()

'{"result": [0, 0]}'

TODO: In the cell below, print the logs of the web service and delete the service

In [21]:
service.get_logs()

'2021-03-31T16:41:45,895922900+00:00 - iot-server/run \n2021-03-31T16:41:45,894883500+00:00 - gunicorn/run \n2021-03-31T16:41:45,914095400+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-03-31T16:41:45,943850500+00:00 - rsyslog/run 

In [37]:
service.delete()